In [48]:
import matplotlib.pylab as plt
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np 
import cv2
import os
import csv
import xgboost as xgb
import pandas as pd

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.neighbors import KNeighborsClassifier
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, GridSearchCV
from tensorflow.keras.utils import Sequence

In [58]:
 def retrain_model(x,y):
    url="https://tfhub.dev/google/imagenet/inception_v3/feature_vector/5"
    base_model = hub.KerasLayer(url, input_shape=(200,200,3))

    #base_model.trainable = False #Freezing layers
    model = keras.Sequential([
        base_model,
        layers.Dense(128,activation='relu'),
        layers.Dense(1, activation='softmax')
    ])
    
    model.compile(
        tf.keras.optimizers.Adam(),
        loss='categorical_crossentropy',
        metrics=['accuracy']) 
    
    model.fit(x,y, epochs=15, verbose=2)
    
    return model

In [34]:
def convert_to_tensor(arg):
  arg = tf.convert_to_tensor(arg, dtype=tf.float32)
  return arg

In [21]:
df=pd.read_csv('../recognition/ids.csv', names=['image','class'], header=None)

def read_images_labels(directory,traintest):
    images=[]
    labels=[]
    for f in os.listdir(directory):                                 
        img=cv2.imread(directory+"/"+f , cv2.IMREAD_COLOR)
        img_resized = cv2.resize(img, (200, 200), interpolation = cv2.INTER_LINEAR)
        images.append(img_resized)   
        #print(img)
        #cv2.imshow('image',img)
        #cv2.waitKey(0)
        labels.append(list(df[df.image==traintest+'/'+f]['class'])[0])
    return images, labels

In [10]:
max(df['class'])

100

In [53]:
images_train, images_train_labels =read_images_labels('../../dataset_cropped/train_cropped','train')
images_test, images_test_labels=read_images_labels('../../dataset_cropped/test_cropped','test')        

images_train=np.array(list(images_train))/255
images_test=np.array(list(images_test))/255

images_train_labels=np.asarray(images_train_labels)
images_test_labels=np.asarray(images_test_labels)

images_train = np.asarray(images_train)
images_test = np.asarray(images_test)

In [59]:
model=retrain_model(images_train, images_train_labels)

Epoch 1/15
24/24 - 22s - loss: 0.0000e+00 - accuracy: 0.0120 - 22s/epoch - 908ms/step
Epoch 2/15
24/24 - 18s - loss: 0.0000e+00 - accuracy: 0.0120 - 18s/epoch - 765ms/step
Epoch 3/15
24/24 - 19s - loss: 0.0000e+00 - accuracy: 0.0120 - 19s/epoch - 774ms/step
Epoch 4/15
24/24 - 20s - loss: 0.0000e+00 - accuracy: 0.0120 - 20s/epoch - 824ms/step
Epoch 5/15
24/24 - 42s - loss: 0.0000e+00 - accuracy: 0.0120 - 42s/epoch - 2s/step
Epoch 6/15
24/24 - 25s - loss: 0.0000e+00 - accuracy: 0.0120 - 25s/epoch - 1s/step
Epoch 7/15
24/24 - 21s - loss: 0.0000e+00 - accuracy: 0.0120 - 21s/epoch - 859ms/step
Epoch 8/15
24/24 - 19s - loss: 0.0000e+00 - accuracy: 0.0120 - 19s/epoch - 798ms/step
Epoch 9/15
24/24 - 19s - loss: 0.0000e+00 - accuracy: 0.0120 - 19s/epoch - 778ms/step
Epoch 10/15
24/24 - 21s - loss: 0.0000e+00 - accuracy: 0.0120 - 21s/epoch - 873ms/step
Epoch 11/15
24/24 - 21s - loss: 0.0000e+00 - accuracy: 0.0120 - 21s/epoch - 891ms/step
Epoch 12/15
24/24 - 22s - loss: 0.0000e+00 - accuracy: 0.0

In [60]:
y_pred=(model.predict(images_test))

In [28]:
print(images_train.shape)
print(y_train.shape)

(750, 200, 200, 3)
(750, 101)
